In [1]:
# import the necessary packages
import numpy as np
import imutils
import cv2
import os
import skimage
import PIL
import matplotlib
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from imutils import paths

In [2]:
# resize the image to a fixed size, then flatten the image into
# a list of raw pixel intensities
def image_to_feature_vector (image,size=(32,32)):
    return cv2.resize(image, size).flatten()

# extract a 3D color histogram from the HSV color space using
# the supplied number of `bins` per channel
# hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
def extract_color_histogram(image,bins=(8,8,8)):
    hist=cv2.calcHist(image, [0,1,2], None, bins, [0,180,0,256,0,256])
 
    if imutils.is_cv2(): # handle normalizing the histogram if we are using OpenCV 2.4.X
        hist=cv2.normalize(hist)
    else :# otherwise, perform "in place" normalization in OpenCV 3
        cv2.normalize(hist,hist)
        return hist.flatten() # return the flattened histogram as the feature vector

In [3]:
# grab the list of images that we'll be describing
# print("[INFO] describing images...")
imagePaths = list(paths.list_images("C:\\Users\\Sarah\\Documents\\GitHub\\Vega\\train_vega"))

In [4]:
# initialize the raw pixel intensities matrix, the features matrix,
# and labels list
rawImages = []
features = []
distortion = []
labels = []

In [5]:
# loop over the input images
for (i, imagePath) in enumerate(imagePaths):
# load the image and extract the class label (assuming that our
# path as the format: /path/to/dataset/{class}.{image_num}.jpg
    image = cv2.imread(imagePath)
    image_flip = cv2.flip(image,1)
    for i in range(1, 11):
        image_dist = np.uint8(image + i*0.1 * image.std() * np.random.random(image.shape))
        label = imagePath.split(os.path.sep)[-1].split(".")[0]
    
# extract raw pixel intensity "features", followed by a color
# histogram to characterize the color distribution of the pixels
# in the image
    pixels = image_to_feature_vector(image)
    pixels_flip = image_to_feature_vector(image_flip)
    pixels_dist = image_to_feature_vector(image_dist)
    hist = extract_color_histogram(image)
    hist_flip = extract_color_histogram(image_flip)
    hist_dist = extract_color_histogram(image_dist)

# update the raw images, features and labels matricies,
# respectively
    rawImages.append(pixels)
    rawImages.append(pixels_flip)
    rawImages.append(pixels_dist)
    features.append(hist)
    features.append(hist_flip)
    features.append(hist_dist)
    labels.append(label)
    labels.append(label)
    labels.append(label)

# show an update every 1,000 images
    if i > 0 and i % 1000 == 0:
        print("[INFO] processed {}/{}".format(i, len(imagePaths)))

# show some information on the memory consumed by the raw images
# matrix and features matrix
rawImages = np.array(rawImages)
features = np.array(features)
labels = np.array(labels)
print("[INFO] pixels matrix: {:.2f}MB".format(
    rawImages.nbytes / (1024 * 1000.0)))
print("[INFO] features matrix: {:.2f}MB".format(
    features.nbytes / (1024 * 1000.0)))

[INFO] pixels matrix: 1.01MB
[INFO] features matrix: 0.67MB


In [6]:
# partition the data into training and testing splits, using 75%
# of the data for training and the remaining 25% for testing
(trainRI, testRI, trainRL, testRL) = train_test_split(
    rawImages, labels, test_size=0.25, random_state=42)
(trainFeat, testFeat, trainLabels, testLabels) = train_test_split(
    features, labels, test_size=0.25, random_state=42)

In [7]:
svm_kernel = ["linear","poly","rbf","sigmoid"]
# train and evaluate a SVM classifer on the raw pixel intensities
print("[INFO] evaluating raw pixel accuracy...")
for k in range(len(svm_kernel)) :
    model = SVC(kernel=svm_kernel[k], C=1, random_state=0)
    model.fit(trainRI, trainRL)
    RL_pred = model.predict(testRI)
    acc = model.score(testRI, testRL)
    print("[INFO] raw pixel accuracy: {:.2f}%".format(acc * 100))
    print(RL_pred)

[INFO] evaluating raw pixel accuracy...
[INFO] raw pixel accuracy: 69.05%
['bouche' 'Plume' 'skara' 'Plume' 'oeil' 'Ankh' 'Plume' 'skara' 'skara'
 'hibou' 'Plume' 'Plume' 'Ankh' 'bouche' 'hibou' 'oeil' 'lapin' 'lapin'
 'hibou' 'skara' 'Plume' 'hibou' 'skara' 'skara' 'Ankh' 'skara' 'Ankh'
 'Plume' 'hibou' 'bouche' 'skara' 'Plume' 'skara' 'skara' 'hibou' 'bouche'
 'lapin' 'Plume' 'bouche' 'skara' 'bouche' 'Ankh' 'skara' 'Plume' 'hibou'
 'Plume' 'skara' 'skara' 'skara' 'Ankh' 'Plume' 'skara' 'oeil' 'oeil'
 'Plume' 'skara' 'Plume' 'skara' 'skara' 'Ankh' 'Ankh' 'Plume' 'bouche'
 'skara' 'skara' 'Ankh' 'bouche' 'Plume' 'Ankh' 'Plume' 'skara' 'skara'
 'Ankh' 'Ankh' 'hibou' 'hibou' 'skara' 'oeil' 'bouche' 'hibou' 'skara'
 'oeil' 'hibou' 'hibou']
[INFO] raw pixel accuracy: 69.05%
['bouche' 'Plume' 'skara' 'Plume' 'oeil' 'Ankh' 'Plume' 'skara' 'skara'
 'hibou' 'Plume' 'Plume' 'Ankh' 'bouche' 'hibou' 'oeil' 'lapin' 'lapin'
 'hibou' 'skara' 'Plume' 'hibou' 'skara' 'skara' 'Ankh' 'Ankh' 'Ankh'
 'Pl

In [8]:
# train and evaluate a SVM classifer on the histogram
# representations
print("[INFO] evaluating histogram accuracy...")
for k in range(len(svm_kernel)) :
    model = SVC(kernel=svm_kernel[k], C=1, random_state=0)
    model.fit(trainFeat, trainLabels)
    Labels_pred = model.predict(testFeat)
    acc = model.score(testFeat, testLabels)
    print("[INFO] histogram accuracy: {:.2f}%".format(acc * 100))
    print(Labels_pred)

[INFO] evaluating histogram accuracy...
[INFO] histogram accuracy: 15.48%
['Plume' 'Plume' 'Plume' 'Plume' 'Plume' 'Plume' 'Plume' 'Plume' 'Plume'
 'Plume' 'Plume' 'Plume' 'Plume' 'Plume' 'Plume' 'Plume' 'Plume' 'Plume'
 'Plume' 'Plume' 'Plume' 'Plume' 'Plume' 'Plume' 'Plume' 'Plume' 'Plume'
 'Plume' 'Plume' 'Plume' 'Plume' 'Plume' 'Plume' 'Plume' 'Plume' 'Plume'
 'Plume' 'Plume' 'Plume' 'Plume' 'Plume' 'Plume' 'Plume' 'Plume' 'Plume'
 'Plume' 'Plume' 'Plume' 'Plume' 'Plume' 'Plume' 'Plume' 'Plume' 'Plume'
 'Plume' 'Plume' 'Plume' 'Plume' 'Plume' 'Plume' 'Plume' 'Plume' 'Plume'
 'Plume' 'Plume' 'Plume' 'Plume' 'Plume' 'Plume' 'Plume' 'Plume' 'Plume'
 'Plume' 'Plume' 'Plume' 'Plume' 'Plume' 'Plume' 'Plume' 'Plume' 'Plume'
 'Plume' 'Plume' 'Plume']
[INFO] histogram accuracy: 15.48%
['Plume' 'Plume' 'Plume' 'Plume' 'Plume' 'Plume' 'Plume' 'Plume' 'Plume'
 'Plume' 'Plume' 'Plume' 'Plume' 'Plume' 'Plume' 'Plume' 'Plume' 'Plume'
 'Plume' 'Plume' 'Plume' 'Plume' 'Plume' 'Plume' 'Plume' 'Plume

In [9]:
from sklearn import metrics
#from sklearn.model_selection import cross_val_score
from sklearn.cross_validation import cross_val_score
from sklearn.cross_validation import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score

# evaluation modèle SVM
print (metrics.classification_report(testRL, RL_pred))
print (metrics.classification_report(testLabels, Labels_pred))

             precision    recall  f1-score   support

       Ankh       0.00      0.00      0.00        15
      Plume       0.15      1.00      0.27        13
     bouche       0.00      0.00      0.00        11
      hibou       0.00      0.00      0.00        17
      lapin       0.00      0.00      0.00         3
       oeil       0.00      0.00      0.00         9
      skara       0.00      0.00      0.00        16

avg / total       0.02      0.15      0.04        84

             precision    recall  f1-score   support

       Ankh       0.00      0.00      0.00        15
      Plume       0.15      1.00      0.27        13
     bouche       0.00      0.00      0.00        11
      hibou       0.00      0.00      0.00        17
      lapin       0.00      0.00      0.00         3
       oeil       0.00      0.00      0.00         9
      skara       0.00      0.00      0.00        16

avg / total       0.02      0.15      0.04        84



C:\Users\Sarah\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\Users\Sarah\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
